In [1]:
import json

from sources.scienceworld import parse_beliefs, parse_goal

from sources.cl_nli.model import SimCSE
from transformers import AutoTokenizer



In [2]:


goldpath_file = "/opt/data/scienceworld-goldpaths/goldsequences-0.json"

with open(goldpath_file) as f:
    json_data = json.load(f)

json_data = json_data['0']

json_data.keys()

dict_keys(['taskIdx', 'taskName', 'goldActionSequences'])

In [3]:
trajectories = json_data['goldActionSequences'][0]['path']
goal = json_data['goldActionSequences'][0]['taskDescription']

all_lens = []

last_reward = 0
for i, trajectory in enumerate(trajectories):
    observation = trajectory['observation']
    look_around = trajectory['freelook']
    inventory = trajectory['inventory']
    
    belief_base = parse_beliefs(observation=observation, look=look_around, inventory=inventory)
    all_lens.append(len(belief_base))
    action = trajectory['action']
    reward =  float(trajectory['score']) - last_reward
    last_reward = float(trajectory['score'])
    is_done =  trajectory['isCompleted']
    if is_done == 'true':
        next_state = ""
        print("finish")
        break
        # ou break de repente aqui
    print(f"Step {i} - reward: {reward:.3f} - is_done: {is_done} - score {float(trajectory['score']):.3f}")

Step 0 - reward: 0.000 - is_done: false - score 0.000
Step 1 - reward: 0.000 - is_done: false - score 0.000
Step 2 - reward: 0.000 - is_done: false - score 0.000
Step 3 - reward: 0.000 - is_done: false - score 0.000
Step 4 - reward: 0.000 - is_done: false - score 0.000
Step 5 - reward: 0.000 - is_done: false - score 0.000
Step 6 - reward: 0.000 - is_done: false - score 0.000
Step 7 - reward: 0.000 - is_done: false - score 0.000
Step 8 - reward: 0.000 - is_done: false - score 0.000
Step 9 - reward: 0.033 - is_done: false - score 0.033
Step 10 - reward: 0.000 - is_done: false - score 0.033
Step 11 - reward: 0.000 - is_done: false - score 0.033
Step 12 - reward: 0.667 - is_done: false - score 0.700
Step 13 - reward: 0.000 - is_done: false - score 0.700
Step 14 - reward: 0.000 - is_done: false - score 0.700
Step 15 - reward: 0.017 - is_done: false - score 0.717
Step 16 - reward: 0.017 - is_done: false - score 0.733
Step 17 - reward: 0.000 - is_done: false - score 0.733
Step 18 - reward: 0.

In [6]:
import numpy as np

np.mean(all_lens)

21.243243243243242

In [4]:



ckpt = "/opt/models/simcse_default/version_0/v0-epoch=4-step=18304-val_nli_loss=0.658-train_loss=0.551.ckpt"

model: SimCSE = SimCSE.load_from_checkpoint(ckpt).eval()
hf_model_name = model.hparams['hf_model_name']
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)

/home/ichida/miniconda3/envs/default_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model

SimCSE(
  (backbone_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [1]:
import torch

belief_base = torch.randn((32,32,768), device='cuda')

belief_base

tensor([[[-0.2512, -0.0770, -1.8312,  ..., -0.1948,  0.6119,  0.3693],
         [-0.5452, -1.2114, -0.8738,  ..., -1.3984, -1.0369,  0.6169],
         [-0.4250,  0.9773,  0.1690,  ...,  1.3402,  0.9726,  0.3732],
         ...,
         [ 0.1894, -0.5548,  0.7270,  ..., -0.9249,  1.3127, -0.4603],
         [-0.6409,  0.5757, -1.4390,  ...,  1.3706, -0.5904,  0.6554],
         [ 0.4480,  1.4759, -0.1090,  ...,  0.1703, -1.0066,  0.9359]],

        [[-1.8288,  0.4786, -0.8813,  ..., -0.7161,  0.8268, -0.1945],
         [-0.2015,  0.1621, -0.0977,  ..., -0.0881, -0.4077,  0.1681],
         [ 0.3353, -0.2826,  1.1525,  ...,  0.4879,  0.2841,  0.6778],
         ...,
         [-0.2653, -0.4667,  1.0247,  ...,  0.8670, -0.0372, -0.0355],
         [-1.3333,  1.7575,  0.5449,  ...,  0.3856,  0.2825, -0.1094],
         [-2.6413, -0.7229, -0.5712,  ..., -1.4301, -0.2122,  0.2355]],

        [[-0.2935,  0.4348,  0.1315,  ...,  0.0426, -0.3539, -0.8621],
         [ 2.8826,  0.1030, -0.3093,  ...,  1

In [4]:
torch.cuda.memory_summary(abbreviated=True)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   3072 KiB |   3078 KiB |   3109 KiB |  38400 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |   3072 KiB |   3078 KiB |   3109 KiB |  38400 B   |\n|---------------------------------------------------------------------------|\n| Requested memory      |   3072 KiB |   3076 KiB |   3082 KiB |  10524 B   |\n|--------------------------------------------------